In [2]:
pip install kaggle

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
#API to fetch the dataset
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 90% 73.0M/80.9M [00:01<00:00, 61.0MB/s]
100% 80.9M/80.9M [00:01<00:00, 55.2MB/s]


In [8]:
#extracting the compressed dataset
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('the dataset is extracted')

the dataset is extracted


IMPORT THE DEPENDENCIES


In [12]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
#print the stopwords
print (stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

DATA processing


In [15]:
twitter_data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv",encoding = 'ISO-8859-1')

checking the no of rows and columns

In [17]:
twitter_data.shape

(1599999, 6)

In [18]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [19]:
twitter_data.tail()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599998,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


naming the columns and read the again

In [20]:
column_names =['target','id','date','flag','user','text']
twitter_data =pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names,encoding='ISO-8859-1')

In [21]:
twitter_data.shape

(1600000, 6)

In [22]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


missing value

In [23]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

checking the distribution of target column

In [24]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

convert the target 4 to 1

In [28]:
twitter_data.replace({'target':{4:1}},inplace= True)

In [29]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 -------> negative tweet
1 -------> positive tweet
STEMMING


In [32]:
#STEMMING IS THE PROCESS OF REDUCING A WORD TO ITS ROOT WORD
  # EG : actor =act
port_stem =PorterStemmer()

In [ ]:
def stemming(content):

  stemmed_content = re.sub('[a-zA-Z]','',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '. join(stemmed_content)
  return stemmed_content

In [49]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [50]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@ ://./21 - , ' . . ;"
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,' ... . !
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@ . 50%
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","@ , ' . ' . ? ' ."


In [51]:
print(twitter_data['stemmed_content'])

0          @ ://./21 - , ' . . ;
1                      ' ... . !
2                        @ . 50%
3                               
4              @ , ' . ' . ? ' .
                   ...          
1599995                        .
1599996        . - ! â« ://./~8
1599997                        ?
1599998                   38 !!!
1599999                 # @ @ @4
Name: stemmed_content, Length: 1600000, dtype: object


In [52]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


separating the data and label

In [53]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [55]:
print(X)

["@ ://./21 - , ' . . ;" "' ... . !" '@ . 50%' ... '?' '38 !!!' '# @ @ @4']


In [56]:
print(Y)

[0 0 0 ... 1 1 1]


splitting the data to training data and test data

In [59]:
x_train,x_test,y_train,y_test = train_test_split(X, Y,test_size=0.2,stratify=Y,random_state=2)

In [61]:
print(x_test)

["@ - ! ' . &; ." '/ &;' '@ !' ... '@41 ! 4700 !' '' '@ !!!! !']


converting the textual data to numerical data

In [69]:
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [70]:
print(x_train)

  (5, 3824)	1.0
  (6, 9711)	0.7382151603694854
  (6, 9671)	0.6745653244872989
  (9, 6023)	1.0
  (20, 2605)	1.0
  (24, 7115)	1.0
  (29, 811)	1.0
  (30, 589)	1.0
  (35, 1053)	1.0
  (45, 1645)	1.0
  (46, 812)	1.0
  (61, 2461)	0.7764611878865577
  (61, 811)	0.6301650765520062
  (74, 6533)	1.0
  (87, 6023)	1.0
  (88, 7168)	1.0
  (96, 4686)	0.59751563005409
  (96, 5007)	0.8018572640071696
  (99, 4486)	1.0
  (106, 726)	1.0
  (107, 2835)	1.0
  (108, 4785)	1.0
  (109, 1)	0.6523247064850503
  (109, 2023)	0.7579396264275887
  (110, 7749)	1.0
  :	:
  (1279841, 9445)	1.0
  (1279842, 6757)	1.0
  (1279843, 6430)	1.0
  (1279848, 861)	1.0
  (1279849, 3491)	1.0
  (1279851, 3913)	1.0
  (1279861, 1879)	1.0
  (1279863, 4978)	1.0
  (1279864, 2992)	1.0
  (1279880, 2993)	1.0
  (1279894, 1371)	1.0
  (1279900, 5050)	1.0
  (1279913, 5133)	1.0
  (1279914, 3389)	1.0
  (1279917, 811)	1.0
  (1279934, 5832)	0.8171898843192867
  (1279934, 8476)	0.5763685391884528
  (1279935, 1459)	1.0
  (1279942, 9086)	1.0
  (1279959,

In [71]:
print(x_test)

  (6, 8026)	1.0
  (15, 2763)	1.0
  (19, 1821)	1.0
  (20, 7630)	1.0
  (23, 3915)	1.0
  (37, 2763)	1.0
  (41, 7959)	1.0
  (45, 9086)	1.0
  (54, 981)	1.0
  (61, 6929)	1.0
  (64, 811)	1.0
  (66, 3983)	0.7133601647015058
  (66, 2835)	0.45594085008270935
  (66, 428)	0.5321984748596118
  (78, 1702)	1.0
  (92, 7597)	1.0
  (110, 9495)	1.0
  (113, 1187)	1.0
  (114, 4486)	1.0
  (115, 3489)	1.0
  (123, 3913)	1.0
  (131, 9495)	1.0
  (149, 5199)	1.0
  (170, 812)	1.0
  (176, 812)	1.0
  :	:
  (319834, 3913)	1.0
  (319844, 2803)	1.0
  (319849, 7804)	1.0
  (319851, 4172)	1.0
  (319860, 2803)	0.6726692776954413
  (319860, 0)	0.7399432700178394
  (319862, 7168)	1.0
  (319878, 4685)	0.7751784803076076
  (319878, 3913)	0.6317422921318377
  (319879, 7265)	1.0
  (319887, 811)	1.0
  (319898, 3650)	1.0
  (319907, 3658)	1.0
  (319917, 3888)	1.0
  (319922, 6033)	1.0
  (319929, 897)	1.0
  (319941, 9425)	1.0
  (319952, 2042)	1.0
  (319961, 7512)	1.0
  (319967, 4172)	1.0
  (319970, 1645)	1.0
  (319982, 3489)	1.0
  (

logistic regession

In [72]:
model = LogisticRegression(max_iter=1000)


In [74]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

In [76]:
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(y_train,x_train_prediction)

In [77]:
print('accuracy score on the training data :',training_data_accuracy)

accuracy score on the training data : 0.5200375


In [80]:
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(y_test,x_test_prediction)

In [81]:
print('accuracy score on the training data :',test_data_accuracy)

accuracy score on the training data : 0.51445


model accuracy = 51.4%

In [82]:
#saving the trained model
import pickle

In [83]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

using the saved model for future prediction

In [85]:
#loading the saved model
loaded_model =pickle.load(open('/content/trained_model.sav','rb'))



In [89]:
x_new = x_test[200]
print(y_test[200])
prediction = model.predict(x_new)
print(prediction)
if(prediction[0]==0):
  print('negative tweet')
else:
  print('positive tweet')


1
[0]
negative tweet
